Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_base.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger base class.

# Library imports.
from binance.client import Client
from abc import abstractmethod, ABC
from time import sleep, time
from os.path import exists, join
from os import mkdir

import pandas as pd

class Crypto_logger_base(ABC):
    def __init__(self, interval='1min', delay=4.7, buffer_size=20000, directory='crypto_logs', 
                 log_name='crypto_log', second_screener=False, raw=False, precise=True):
        """
        :param interval: OHLCV interval to log. Default is 1 minute.
        :param delay: delay between Binance API requests. Minimum calculated was 5 seconds.
        :param buffer_size: buffer size to avoid crashing on low memory.
        :param directory: the directory where to output the logs.
        :param log_name: name of the log file.
        :param raw: whether the log dumps raw (instantaneous) or OHLCV data.
        """
        self.interval = interval
        self.delay = delay
        self.buffer_size = buffer_size
        self.directory = directory
        self.raw = raw
        self.precise = precise

        self.log_name = join(self.directory, log_name + '.txt')
        self.log_screened_name = join(self.directory, log_name + '_screened.txt')
        self.log_screened_2_name = join(self.directory, log_name + '_screened_2.txt')
        self.second_screener = second_screener and 'output' in self.log_screened_2_name

        if not exists(self.directory):
            mkdir(self.directory)

    @abstractmethod
    def screen(self, **kwargs):
        raise NotImplementedError()

    #@abstractmethod
    #def screen_2(self, **kwargs):
    #    raise NotImplementedError()

    @abstractmethod
    def get(self, **kwargs):
        raise NotImplementedError()

    def recalculate_volumes(self, df, df_old):
        df.columns = df.columns.swaplevel(0, 1)
        df_old.columns = df_old.columns.swaplevel(0, 1)
        df = pd.concat([df, df_old[['base_volume', 'quote_volume']]], 
                       join='outer', axis='columns')
        print(df_old.columns)
        df.columns = df.columns.swaplevel(0, 1)
        df = df.stack(level=1).reset_index(level=1)
        df[df['pair'] == 'base_volume'].iloc[:,1:] = \
            df[df['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
            df[df['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
        df[df['pair'] == 'quote_volume'].iloc[:,1:] = \
            df[df['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
            df[df['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
        df = df.reset_index().pivot_table(index=['date'], columns=['pair'], 
                                          values=df.columns[1:])
        return df[['open', 'high', 'low', 'close', 
                   'base_volume', 'quote_volume', 
                   'rolling_base_volume', 
                   'rolling_quote_volume']]

    def resample(self, df):
        df.index = pd.DatetimeIndex(df.index).round(self.interval)
        df = df.stack(level=0).reset_index(level=1)
        frequency = pd.tseries.frequencies.to_offset((df.index[1:] - df.index[:-1]).min())
        frequency_1min = pd.tseries.frequencies.to_offset('1min')
        if frequency > frequency_1min:
            df = df.pivot_table(index=['date'], columns=['symbol'], 
                                values=['open', 'high', 'low', 'close', 
                                        'base_volume', 'quote_volume', 
                                        'rolling_base_volume', 
                                        'rolling_quote_volume'], 
                                aggfunc={'open': 'first', 'high': 'max', 
                                         'low': 'min', 'close': 'last', 
                                         'base_volume': 'sum', 
                                         'quote_volume': 'sum', 
                                         'rolling_base_volume': 'sum', 
                                         'rolling_quote_volume': 'sum'})
        else:
            df = df.pivot_table(index=['date'], columns=['symbol'], 
                                values=['open', 'high', 'low', 'close', 
                                        'rolling_base_volume', 
                                        'rolling_quote_volume'], 
                                aggfunc={'open': 'first', 'high': 'max', 
                                         'low': 'min', 'close': 'last', 
                                         'rolling_base_volume': 'max', 
                                         'rolling_quote_volume': 'max'})

        if frequency == frequency_1min:
            df = recalculate_volumes(df, self.dataset)

        df.columns = df.columns.swaplevel(0, 1)
        return df.sort_index()

    def put(self, dataset):
        dataset = dataset.copy().reset_index()
        if self.raw:
            dataset = dataset.drop_duplicates(subset=['symbol', 'count'], 
                                              keep='first', ignore_index=True)
        else:
            dataset = dataset.drop_duplicates(keep='last', ignore_index=True)

        if 'date' in dataset.columns:
            min_index_int = dataset[dataset['date'] == self.min_index].index[0]
            dataset = dataset.set_index('date')
        if not self.raw:
            dataset = self.resample(dataset)
        if 'date' in dataset.columns:
            dataset = dataset.iloc[min_index_int:]

        dataset = dataset.tail(self.buffer_size)
        dataset.to_csv(self.log_name)
        self.min_index = dataset.index[0]
        return dataset

    def start(self, append=False, roll=0):
        """Main logger loop."""
        print('Starting crypto logger.')

        if exists(self.log_name) and 'output' in self.log_name:
            self.dataset = pd.read_csv(self.log_name, header=[0, 1], index_col=0)
            self.dataset = self.dataset.sort_index()
        else:
            self.dataset = self.get()

        self.min_index = self.dataset.index[-1]
        self.dataset = self.put(self.dataset)
        if self.precise:
            t1 = time()

        while True:
            try:
                dataset = pd.concat([self.dataset, self.get()], axis='index', join='outer')
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            try:
                self.dataset = self.put(dataset)
            except (KeyboardInterrupt, SystemExit):
                print('Saving latest complete dataset...')
                self.dataset = self.put(dataset)
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            '''
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                if self.second_screener and exists(self.log_screened_2_name):
                    dataset_screened_2_old = \
                        pd.read_csv(self.log_screened_2_name, index_col=0, header=0)
                else:
                    dataset_screened_2_old = None
                dataset_screened = self.screen(self.dataset)
                if self.second_screener:
                    dataset_screened_2 = self.screen_2(dataset_screened)
                if roll != 0:
                    if append and exists(self.log_screened_name):
                        if dataset_screened is not None:
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        if self.second_screener and exists(self.log_screened_2_name) and dataset_screened_2 is not None:
                            dataset_screened_2 = \
                                pd.concat([dataset_screened_2_old, dataset_screened_2], axis='index')
                            dataset_screened_2 = \
                                dataset_screened_2.drop_duplicates(subset=['symbol'], keep='last')
                    if dataset_screened is not None:
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    if self.second_screener and exists(self.log_screened_2_name) and dataset_screened_2 is not None:
                        dataset_screened_2 = dataset_screened_2.tail(roll)
                        dataset_screened_2.to_csv(self.log_screened_2_name)
                elif append:
                    if dataset_screened is not None:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    if self.second_screener and dataset_screened_2 is not None:
                        dataset_screened_2.to_csv(self.log_screened_2_name, mode='a')
                else:
                    if dataset_screened is not None:
                        dataset_screened.to_csv(self.log_screened_name)
                    if self.second_screener and dataset_screened_2 is not None:
                        dataset_screened_2.to_csv(self.log_screened_2_name)
            '''
            try:
                if exists(self.log_screened_name):
                    dataset_screened_old = \
                        pd.read_csv(self.log_screened_name, index_col=0, header=0)
                else:
                    dataset_screened_old = None
                dataset_screened = self.screen(self.dataset)
                if dataset_screened is not None:
                    if roll != 0:
                        if append and exists(self.log_screened_name):
                            dataset_screened = \
                                pd.concat([dataset_screened_old, dataset_screened], axis='index')
                            dataset_screened = \
                                dataset_screened.drop_duplicates(subset=['symbol'], keep='last')
                        dataset_screened = dataset_screened.tail(roll)
                        dataset_screened.to_csv(self.log_screened_name)
                    elif append:
                        dataset_screened.to_csv(self.log_screened_name, mode='a')
                    else:
                        dataset_screened.to_csv(self.log_screened_name)
            except (KeyboardInterrupt, SystemExit):
                print('User terminated crypto logger process.')
                break
            except Exception as e:
                print(e)
            if self.precise:
                t2 = time()
                if t2 - t1 < self.delay:
                    sleep(t2 - t1 + self.delay)
                t1 = t2
            else:
                sleep(self.delay)
        print('Crypto logger process done.')

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.


# Library imports.
from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.renko import get_renko_trigger
from os import mkdir
from os.path import exists, join
from sys import float_info as sflt
from numpy import log
from pandas_ta.utils._core import signed_series, recent_minimum_index

import datetime
import pandas_ta as ta
import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=6, interval_input='15s', interval='15s', buffer_size=100, 
                 input_log_name='input', second_screener=False):
        """
        :param delay: delay between Binance API requests. Minimum calculated was 5 seconds.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on low memory.
        :param directory: the directory where to output the logs.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         second_screener=second_screener, raw=False, precise=True)

        #if not self.load_from_ohlcv:
        #    self.input_log_screened_up_name = \
        #        join(self.directory, input_log_name + interval_input + '_screened_up.txt')

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def get_screened(self, data_after, price_threshold=5.0, volume_threshold=300.0):
        price_movers = pd.DataFrame()
        volume_movers = pd.DataFrame()
        data_before = self.data_before
        if data_before.size != 0:
            data_before.columns = data_before.columns.swaplevel(0, 1)
            data_after.columns = data_after.columns.swaplevel(0, 1)
            price_before = data_before['close'].pct_change(1)
            price_after = data_after['close'].pct_change(1)
            volume_before = data_before['volume'].shift(1)
            volume_after = data_after['volume'].shift(1)
            price_percent_change_before = \
                ((data_before['close'].pct_change(1) - price_before) / price_before)
            price_percent_change_after = \
                ((data_after['close'].pct_change(1) - price_after) / price_after)
            volume_percent_change_before = ((data_before['volume'] - volume_before) / volume_before)
            volume_percent_change_after = ((data_after['volume'] - volume_after) / volume_after)
            price_movers = \
                ((price_percent_change_after - price_percent_change_before) * 100) > price_threshold
            volume_movers = \
                ((volume_percent_change_after - volume_percent_change_before) * 100) > volume_threshold
            price_movers = data_after[price_movers].columns.tolist()
            volume_movers = data_after[volume_movers].columns.tolist()
            data_before.columns = data_before.columns.swaplevel(0, 1)
            data_after.columns = data_after.columns.swaplevel(0, 1)
        self.data_before = data_after
        return price_movers + volume_movers

    def screen(self, dataset):
        def filter_in_market(function, dataset):
            def f(x):
                x = x.loc[:,~x.columns.duplicated()]
                return function(x)
            tickers_list = dataset.columns.get_level_values(0).unique().tolist()
            return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

        def get_relative_volume_levels_smoothed_thresholded(data):
            try:
                volume = data['volume']
                #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
                volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
                #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
                rvol = (volume / volume.shift(1))
                rvol = rvol.fillna(method='pad')
                bar_up = (ticker['close'] > ticker['open'])
                bar_up |= (ticker['close'] == ticker['open']) & (ticker['close'].diff() > 0)
                bar_up = bar_up.astype(int)
                bar_up = bar_up * 2 - 1
                rvol *= bar_up
                rvol_indicator = ta.hma(rvol, length=14, talib=True)
                rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
                #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
                threshold = 2
                rvol_thresholded = (rvol_indicator > threshold).iloc[-1]
            except:
                rvol_thresholded = False
            return rvol_thresholded

        def get_not_square_wave_trigger_1(data):
            return not (data.iloc[-4:]['close'].unique().size < 2)

        def get_not_square_wave_trigger_2(data):
            return not (data.iloc[-15:]['close'].unique().size < 6)

        def get_not_square_wave_trigger_3(data):
            return (data[['open', 'high', 'low', 'close']].nunique(axis='columns') > 2).tail(2).all()

        def get_bullish_price_trigger(data):
            return (data['close'] > data['high'].shift(1)).iloc[-1]

        def get_positive_RSI_trigger(data):
            RSI_6 = data.ta.rsi(length=6, talib=True)
            RSI_12 = data.ta.rsi(length=12, talib=True)
            RSI_24 = data.ta.rsi(length=24, talib=True)
            data = ((RSI_6 > RSI_12) | (RSI_6 > RSI_24) | (RSI_12 > RSI_24))
            return data.iloc[-1]

        def get_positive_momentum_trigger(data):
            KDJ = data.ta.kdj(length=5, signal=3, talib=True)
            return ((KDJ['J_5_3'] > KDJ['D_5_3']) & (KDJ['J_5_3'] > KDJ['K_5_3'])).iloc[-1]

        def get_positive_JMA_trigger(data):
            JMA = data.ta.jma(length=7, phase=0, talib=True)
            return (data['close'] < JMA).iloc[-1]

        def get_ease_of_movement(data):
            eom = ((data['high'] - data['low']) / (2 * data['volume'] + 1))
            eom *= (data['high'].diff(1) + data['low'].diff(1))
            precision = eom.abs().max()
            if precision < 1:
                eom *= 1 / precision
            return eom

        def get_ease_of_movement_trigger(data):
            data[['open', 'high', 'low', 'close']] += sflt.epsilon
            data[['volume']] += 1

            log_price = log(data['close'])
            price_trough_index = recent_minimum_index(signed_series(log_price, initial=None))
            price_slope = ta.slope(close=log_price, length=price_trough_index, as_angle=True, 
                                   to_degrees=True, talib=True)

            EOM = get_ease_of_movement(data)
            EOM_trough_index = recent_minimum_index(signed_series(EOM, initial=None))
            EOM_slope = ta.slope(close=EOM, length=EOM_trough_index, as_angle=True, 
                                 to_degrees=True, talib=True)

            trigger = (price_slope <= EOM_slope)
            #trigger &= (EOM_slope >= 0.25)
            trigger &= (EOM_slope > 0.0)

            #breakout_trigger = ((EOM.shift(1) < 0.0) & (EOM > 0.0))
            #trigger |= breakout_trigger
            return trigger.iloc[-1]

        '''
        def get_positive_PVR_trigger(data):
            price = data['close']
            volume = data['volume']
            volume.iloc[-1] *= volume_multiplier
            price_trigger = (price.diff() > 0)
            volume_trigger = (volume.diff() > 0)
            trigger = (price_trigger & volume_trigger)
            return trigger.iloc[-1]
        '''

        def get_rising_volume_trigger(data):
            return (data['rolling_base_volume'].diff(1) > 0).iloc[-1]

        def get_RSI_reversal_trigger(data, rsi_length=2, upper_threshold=95, 
                                     lower_threshold=5, positive=True):
            RSI = data.ta.rsi(length=rsi_length, talib=True)
            RSI_prev = RSI.shift(1)
            thresholds_bear = -((RSI_prev >= upper_threshold) & (RSI < upper_threshold)).astype(int)
            thresholds_bull = ((RSI_prev <= lower_threshold) & (RSI > lower_threshold)).astype(int)
            thresholds = (thresholds_bear + thresholds_bull)
            thresholds = thresholds.replace(to_replace=0, method='pad')
            return (thresholds == (1 if positive else -1)).iloc[-1]

        def get_heikin_ashi_trigger(data):
            def get_positive_trend_strength_trigger(data):
                ADX = data.ta.adx(talib=True)
                return (ADX['ADX_14'] < 0.20).iloc[-3] and (ADX['ADX_14'] > 0.20).iloc[-2]

            def get_not_negative_trend_strength_trigger(data):
                ADX = data.ta.adx(length=14, lensig=8, talib=True)
                return ((ADX['DMP_14'] > ADX['DMN_14']) and (ADX['ADX_14'] > 0.30)).iloc[-1]

            def get_not_negative_rebound_trigger(data):
                CCI = data.ta.cci(length=22, talib=True)
                MFI = data.ta.mfi(length=11, talib=True)
                return ((CCI > 0) or (MFI > 20)).iloc[-1]

            def get_positive_choppiness_trigger(data):
                CHOP = data.ta.chop(talib=True)
                return CHOP.iloc[-1] < 38.2

            def get_positive_phase_trigger(data):
                MACD = data.ta.macd(talib=True)
                histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
                return ((histogram.iloc[-1] > histogram.iloc[-2]) or \
                        (MACD['MACD_12_26_9'].iloc[-1] > MACD['MACDs_12_26_9'].iloc[-1]))

            def get_positive_RSI_trigger(data):
                RSI_5 = data.ta.rsi(length=5, talib=True)
                return ((RSI_5 >= 60) & (RSI_5 <= 65)).iloc[-1]

            def get_negative_PVR_trigger(data):
                price_trigger = (data['close'].iloc[-1] < data['close'].iloc[-2])
                volume_trigger = (data['volume'].iloc[-1] > data['volume'].iloc[-2])
                return price_trigger and volume_trigger

            def get_buy_trigger(data):
                return get_not_negative_rebound_trigger(data) and \
                        (get_positive_choppiness_trigger(data) or \
                        get_positive_trend_strength_trigger(data))

            def get_sell_trigger(data):
                return (((not get_positive_choppiness_trigger(data)) or \
                         get_negative_trend_strength_trigger(data) or \
                         (not get_positive_phase_trigger(data))) or \
                        get_not_negative_rebound_trigger(data))

            heikin_ashi = data.ta.ha(talib=True)
            heikin_ashi_dataset_1 = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                                'HA_high': 'high', 
                                                                'HA_low': 'low', 
                                                                'HA_close': 'close'})
            #heikin_ashi = heikin_ashi_dataset_1.ta.ha(talib=True)
            #heikin_ashi_dataset_2 = heikin_ashi.rename(columns={'HA_open': 'open', 
            #                                                    'HA_high': 'high', 
            #                                                    'HA_low': 'low', 
            #                                                    'HA_close': 'close'})
            #heikin_ashi = heikin_ashi_dataset_2.ta.ha(talib=True)
            #heikin_ashi_dataset_3 = heikin_ashi.rename(columns={'HA_open': 'open', 
            #                                                    'HA_high': 'high', 
            #                                                    'HA_low': 'low', 
            #                                                    'HA_close': 'close'})
            #if get_not_negative_rebound_trigger(heikin_ashi_dataset_1):
            #    return True
            return True \
                if get_positive_phase_trigger(heikin_ashi_dataset_1) \
                else (get_not_negative_rebound_trigger(heikin_ashi_dataset_1) or \
                      get_not_negative_trend_strength_trigger(heikin_ashi_dataset_1))

        def screen_one(pair):
            frequency = pd.tseries.frequencies.to_offset((pair.index[1:] - pair.index[:-1]).min())
            frequency_1min = pd.tseries.frequencies.to_offset('1min')
            frequency_30min = pd.tseries.frequencies.to_offset('30min')
            frequency_1h = pd.tseries.frequencies.to_offset('1h')
            if frequency < frequency_1min:
                pair['volume'] = pair['rolling_base_volume'].copy()
            else:
                pair['volume'] = pair['base_volume'].copy()
            if frequency == frequency_30min:
                if get_not_square_wave_trigger_1(pair):
                    if get_not_square_wave_trigger_2(pair):
                        #if get_bullish_price_trigger(pair):
                        #if get_heikin_ashi_trigger(pair):
                        if get_renko_trigger(pair, compress=False, 
                                             direction_type='long', 
                                             trigger_type='simple', 
                                             method='atr', plot=False):
                            return True
            elif frequency == frequency_1h:
                if get_relative_volume_levels_smoothed_thresholded(pair):
                    return True
            else:
                if get_not_square_wave_trigger_1(pair):
                    if get_not_square_wave_trigger_2(pair):
                        if frequency < frequency_1min:
                            return True 
                        else:
                            if get_rising_volume_trigger(pair):
                                if get_heikin_ashi_trigger(pair):
                                    return True
            return False

        #if not self.load_from_ohlcv:
        #    if exists(self.input_log_screened_up_name):
        #        input_filtered_up = pd.read_csv(self.input_log_screened_up_name, header=0, index_col=None)
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            #if not self.load_from_ohlcv:
            #    input_filter = input_filter & set(input_filtered_up['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]

            #assets = self.get_screened(dataset, price_threshold=1.0, volume_threshold=1.0)
            #input_filtered_movers = input_filtered[input_filtered['symbol'].isin(assets)]
            #input_filtered_movers.to_csv(self.input_log_screened_name, mode='a')
            assets = filter_in_market(screen_one, dataset)
            return input_filtered[input_filtered['symbol'].isin(assets)]
        else:
            return None

    def screen_(self, dataset):
        return dataset

    def resample_from_raw(self, df):
        df = df[['symbol', 'lastPrice', 'volume', 'quoteVolume']]
        df = df.rename(columns={'lastPrice': 'close', 
                                'volume': 'rolling_base_volume', 
                                'quoteVolume': 'rolling_quote_volume'})
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                               names=(None, 'symbol'))
        df = df.rename(columns={'close_first': 'open', 
                                'close_max': 'high', 
                                'close_min': 'low', 
                                'close_last': 'close', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
        df['rolling_base_volume'].iloc[0] = 0
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
        df['rolling_quote_volume'].iloc[0] = 0
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        if self.load_from_ohlcv:
            dataset = pd.read_csv(self.input_log_name, header=[0, 1], index_col=0)
        else:
            dataset = pd.read_csv(self.input_log_name, header=0, index_col=0)
            dataset = self.resample_from_raw(dataset)
        return dataset.sort_index().tail(2)

Main.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_output_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_output import Crypto_logger_output

'''
crypto_logger_output_1min = Crypto_logger_output(delay=8, interval_input='1min', 
                                                 interval='1min', buffer_size=100, 
                                                 input_log_name='input')
crypto_logger_output_1min.start(append=False, roll=1000)
'''

crypto_logger_output_1min = Crypto_logger_output(delay=8, interval_input='30s', 
                                                 interval='1min', buffer_size=2000, 
                                                 input_log_name='output')
crypto_logger_output_1min.start(append=False, roll=1000)

In [ ]:
dataset = crypto_logger_output_1min.dataset
dataset

In [ ]:
dataset['BTCUSDT']

In [ ]:
dataset['ZILBUSD']

In [ ]:
def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

def get_positive_trend_strength_trigger(data):
    ADX = data.ta.adx(talib=True)
    return (ADX['ADX_14'] < 0.20).iloc[-3] & (ADX['ADX_14'] > 0.20).iloc[-2]

def get_not_negative_trend_strength_trigger(data):
    ADX = data.ta.adx(length=14, lensig=8, talib=True)
    return ((ADX['DMP_14'] > ADX['DMN_14']) & (ADX['ADX_14'] > 0.30)).iloc[-1]

def get_not_negative_rebound_trigger(data):
    CCI = data.ta.cci(length=22, talib=True)
    MFI = data.ta.mfi(length=11, talib=True)
    return ((CCI > 0) | (MFI > 20)).iloc[-1]

def get_positive_choppiness_trigger(data):
    CHOP = data.ta.chop(talib=True)
    return CHOP.iloc[-1] < 38.2

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram > histogram.shift(1)) | \
            (MACD['MACD_12_26_9'] > MACD['MACDs_12_26_9'])).iloc[-1]

def get_positive_phase_trigger(data):
    MACD = data.ta.macd(talib=True)
    histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
    return ((histogram.iloc[-2] > histogram.iloc[-2]) or \
            (MACD['MACD_12_26_9'].iloc[-1] > MACD['MACDs_12_26_9'].iloc[-1]))

In [ ]:
filter_in_market(get_positive_phase_trigger, df)

In [ ]:
get_positive_phase_trigger(df['BTCUSDT'])

In [ ]:
from tqdm import tqdm

def filter_in_market(function, dataset):
    def f(x):
        x = x.loc[:,~x.columns.duplicated()]
        return function(x)
    tickers_list = dataset.columns.get_level_values(0).unique().tolist()
    return pd.Series([ticker for ticker in tqdm(tickers_list, unit=' ticker') if f(dataset[ticker])], dtype='str')

def get_heikin_ashi_trigger(ticker):
    def get_trend_strength_entry_trigger(heikin_ashi_dataset):
        ADX = heikin_ashi_dataset.ta.adx(talib=True)
        return (ADX['ADX_14'] < 0.20).iloc[-3] and (ADX['ADX_14'] > 0.20).iloc[-2]

    def get_not_negative_rebound_trigger(heikin_ashi_dataset):
        CCI = heikin_ashi_dataset.ta.cci(talib=True)
        return CCI.iloc[-1] < 100

    def get_positive_choppiness_trigger(heikin_ashi_dataset):
        CHOP = heikin_ashi_dataset.ta.chop(talib=True)
        return CHOP.iloc[-1] < 38.2

    def get_buy_trigger(heikin_ashi_dataset):
        return get_not_negative_rebound_trigger(heikin_ashi_dataset) and \
               (get_positive_choppiness_trigger(heikin_ashi_dataset) or \
                get_trend_strength_entry_trigger(heikin_ashi_dataset))

    heikin_ashi = ticker.ta.ha(talib=True)
    heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                      'HA_high': 'high', 
                                                      'HA_low': 'low', 
                                                      'HA_close': 'close'})
    try:
        trigger = get_buy_trigger(heikin_ashi_dataset)
    except:
        trigger = False
    return trigger

tickers_list = filter_in_market(get_heikin_ashi_trigger, dataset)
tickers_list

In [ ]:
heikin_ashi = dataset['BTCUSDT'].ta.ha(talib=True)
heikin_ashi_dataset = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                  'HA_high': 'high', 
                                                  'HA_low': 'low', 
                                                  'HA_close': 'close'})
ADX = heikin_ashi_dataset.ta.adx(talib=True)
ADX[['DMP_14', 'DMN_14']]

In [1]:
symbol = 'NEBLBUSD'

In [ ]:
import pandas as pd
crypto_output_log_1d = 'crypto_logs/crypto_output_log_1d.txt'
df_1d = pd.read_csv(crypto_output_log_1d, header=[0, 1], index_col=0)
df_1d.index = pd.DatetimeIndex(df_1d.index)
df_1d

In [ ]:
import pandas as pd
crypto_output_log_30s = 'crypto_logs/crypto_output_log_30s.txt'
df_30s = pd.read_csv(crypto_output_log_30s, header=[0, 1], index_col=0)
df_30s.index = pd.DatetimeIndex(df_30s.index)
df_30s

In [2]:
import pandas as pd
crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df_15s = pd.read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df_15s.index = pd.DatetimeIndex(df_15s.index)
df_15s

symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,ACABTC,ACABUSD,ACAUSDT,ACHBUSD,ACHUSDT,ACMBTC,ACMBUSD,...,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBNB,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-11 00:00:00,435.8,826.7,2290.9,16.68,0.0,5779.35,0.0,72348.0,0.0,0.0,...,731893.803052,47.183341,14259.137445,5.424846e+06,6.792564e+06,0.0,2.741655,23835.1569,10.447184,9.951928e+05
2022-10-11 11:30:00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,...,734340.702259,46.665334,14278.038162,5.418066e+06,6.817638e+06,0.0,2.720068,23770.8243,10.492221,1.001443e+06
2022-10-11 12:00:00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,...,736617.949352,46.653747,14292.860899,5.413442e+06,6.830076e+06,0.0,2.717524,23809.6313,10.492221,1.003301e+06


In [3]:
df_15s[symbol]

,base_volume,close,high,low,open,quote_volume,rolling_base_volume,rolling_quote_volume
date,,,,,,,,
2022-10-11 00:00:00,1578.1,2.779,2.780,2.774,2.774,4343.165,1293406.6,3.748034e+06
2022-10-11 11:30:00,0.0,2.784,2.784,2.774,2.774,0.000,1284324.2,3.720728e+06
2022-10-11 12:00:00,0.0,2.791,2.791,2.774,2.774,0.000,1285215.9,3.723047e+06


In [5]:
df_15s.resample('15s').agg('max')

symbol,1INCHBTC,1INCHBUSD,1INCHUSDT,ACABTC,ACABUSD,ACAUSDT,ACHBUSD,ACHUSDT,ACMBTC,ACMBUSD,...,ZILBUSD,ZILETH,ZILEUR,ZILTRY,ZILUSDT,ZRXBNB,ZRXBTC,ZRXBUSD,ZRXETH,ZRXUSDT
,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,base_volume,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-10-11 00:00:00,435.8,826.7,2290.9,16.68,0.0,5779.35,0.0,72348.0,0.0,0.0,...,731893.803052,47.183341,14259.137445,5.424846e+06,6.792564e+06,0.0,2.741655,23835.1569,10.447184,9.951928e+05
2022-10-11 00:00:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-11 00:00:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-11 00:00:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-11 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-11 11:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-11 11:59:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
crypto_output_log_1min = 'crypto_logs/crypto_output_log_1min.txt'
df_1min = pd.read_csv(crypto_output_log_1min, header=[0, 1], index_col=0)
df_1min.index = pd.DatetimeIndex(df_1min.index)
df_1min

In [ ]:
df_1min[symbol]

In [ ]:
df_1min[symbol].dropna()

In [ ]:
df = df_1min.copy()
df

In [ ]:
df.columns = df.columns.swaplevel(0, 1)
df = df.stack(level=1).reset_index(level=1)
df_temp = df[['base_volume', 'quote_volume', 
              'rolling_base_volume', 'rolling_quote_volume']]
df_temp[df_temp['pair'] == 'base_volume'].iloc[:,1:] = \
    df_temp[df_temp['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
    df_temp[df_temp['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
df_temp[df_temp['pair'] == 'quote_volume'].iloc[:,1:] = \
    df_temp[df_temp['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
    df_temp[df_temp['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
df = df.reset_index().pivot_table(index=['date'], columns=['pair'], 
                                  values=df.columns[1:])
df

In [ ]:
df = df_1min.copy()
df = df.iloc[-1440:]
df_old = df.copy().iloc[:-10]
df.columns = df.columns.swaplevel(0, 1)
df = df.sort_index(axis='columns').drop(columns=['base_volume', 'quote_volume'])
df.columns = df.columns.swaplevel(0, 1)
df

In [ ]:
df[symbol]

In [ ]:
df = df_1min.copy()
df = df.iloc[-1440:]
df_old = df.copy().iloc[:-10]
df.columns = df.columns.swaplevel(0, 1)
df = df.sort_index(axis='columns').drop(columns=['rolling_base_volume', 'rolling_quote_volume'])
df.columns = df.columns.swaplevel(0, 1)
df

In [ ]:
df.columns

In [ ]:
grouped = df.groupby('pair', axis='columns', level=1)
grouped

In [ ]:
df[symbol]

In [ ]:
grouped = df.groupby('pair', axis='columns').describe()
grouped

In [ ]:
grouped['mean']

In [ ]:
df.groupby(level=0, axis=1).apply(lambda x: x[x.name].apply(lambda x: x, axis=1))

In [ ]:
df.groupby(level='symbol', axis='index')['base_volume']

In [ ]:
df.groupby(level='pair', axis='columns').apply(lambda x: x)

In [ ]:
df.groupby(level=['symbol', 'pair'], axis='columns').agg(lambda x: x)

In [ ]:
df.groupby('pair', axis=1).apply(lambda x: x[x.name == 'base_volume'])

In [ ]:
df.groupby('pair', axis=1, level=1).apply(lambda x: x.name)

In [ ]:
df.loc[:,(slice(None), 'base_volume')]

In [ ]:
df = df_1min.copy()
idx = pd.IndexSlice
#df_base_volume = df.loc[:,idx[:,'base_volume']]
#df_quote_volume = df.loc[:,idx[:,'quote_volume']]
#df_rolling_base_volume = df.loc[:,idx[:,'rolling_base_volume']]
#df_rolling_quote_volume = df.loc[:,idx[:,'rolling_quote_volume']]
df_volume = df.loc[:,idx[:,['base_volume', 'quote_volume', 'rolling_quote_volume', 'rolling_quote_volume']]].copy()
#df_base_volume = df_rolling_base_volume.droplevel(1, axis=1).diff(1) + df_base_volume.droplevel(1, axis=1).shift(1440)
#df_quote_volume = df_rolling_quote_volume.droplevel(1, axis=1).diff(1) + df_quote_volume.droplevel(1, axis=1).shift(1440)
#df = pd.merge(left=df, right=df_rolling_base_volume, how='right')
#df = pd.merge(left=df, right=df_rolling_quote_volume, how='right')
#df.loc[:,(slice(None), 'base_volume')] = df_base_volume
#df.loc[:,(slice(None), 'quote_volume')] = df_quote_volume
#df.loc[:,idx[:,'base_volume']] = \
#    df.loc[:,idx[:,'rolling_base_volume']].diff(1) + \
#    df.loc[:,idx[:,'base_volume']].shift(1440)
#df.loc[:,idx[:,'quote_volume']] = \
#    df.loc[:,idx[:,'rolling_quote_volume']].diff(1) + \
#    df.loc[:,idx[:,'quote_volume']].shift(1440)
#df['BTCUSDT'].tail(10)
df_volume #.loc[:,idx[:,'base_volume']] + df_volume.loc[:,idx[:,'rolling_base_volume']]

In [ ]:
df = df_1min.copy()
t1 = time()
df.iloc[:,df.columns.get_level_values(1) == 'base_volume'] = \
    df.xs('rolling_base_volume', axis=1, level=1).diff(1) + \
    df.xs('base_volume', axis=1, level=1).shift(1440)
df.iloc[:,df.columns.get_level_values(1) == 'quote_volume'] = \
    df.xs('rolling_quote_volume', axis=1, level=1).diff(1) + \
    df.xs('quote_volume', axis=1, level=1).shift(1440)
t2 = time()
print('It took', t2 - t1, 'seconds')
df

In [ ]:
df[symbol]

In [ ]:
from time import time

def recalculate_volumes(df):
    df = df.stack(level=1)
    df.reset_index(level=1, inplace=True)
    df[df['pair'] == 'base_volume'].iloc[:,1:] = \
        df[df['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
        df[df['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
    df[df['pair'] == 'quote_volume'].iloc[:,1:] = \
        df[df['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
        df[df['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
    df.reset_index(inplace=True)
    df = df.pivot_table(index=['date'], columns=['pair'], values=df.columns[1:], sort=False)
    #df.columns = df.columns.swaplevel(0, 1)
    #df = df[['open', 'high', 'low', 'close', 
    #         'base_volume', 'quote_volume', 
    #         'rolling_base_volume', 
    #         'rolling_quote_volume']]
    #df.columns = df.columns.swaplevel(0, 1)
    return df

df = df_1min.copy()
t1 = time()
df = recalculate_volumes(df)
t2 = time()
print('It took', t2 - t1, 'seconds')
df

In [ ]:
df[symbol]

In [ ]:
df1[symbol]

In [ ]:
%%timeit
df = df.stack(level=1).reset_index(level=1)
df[df['pair'] == 'base_volume'].iloc[:,1:] = \
    df[df['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
    df[df['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
df[df['pair'] == 'quote_volume'].iloc[:,1:] = \
    df[df['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
    df[df['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
df = df.reset_index().pivot_table(index=['date'], columns=['pair'], 
                                  values=df.columns[1:])
df.columns = df.columns.swaplevel(0, 1)
df = df[['open', 'high', 'low', 'close', 
         'base_volume', 'quote_volume', 
         'rolling_base_volume', 
         'rolling_quote_volume']]
df.columns = df.columns.swaplevel(0, 1)
df

In [ ]:
df.columns = df.columns.swaplevel(0, 1)
df_old.columns = df_old.columns.swaplevel(0, 1)
df = pd.concat([df, df_old[['base_volume', 'quote_volume']]], 
               join='outer', axis='columns')
df.columns = df.columns.swaplevel(0, 1)
df = df.stack(level=1).reset_index(level=1)
df[df['pair'] == 'base_volume'].iloc[:,1:] = \
    df[df['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
    df[df['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
df[df['pair'] == 'quote_volume'].iloc[:,1:] = \
    df[df['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
    df[df['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
df = df.reset_index().pivot_table(index=['date'], columns=['pair'], 
                                  values=df.columns[1:])
df.columns = df.columns.swaplevel(0, 1)
df = df[['open', 'high', 'low', 'close', 
         'base_volume', 'quote_volume', 
         'rolling_base_volume', 
         'rolling_quote_volume']]
df.columns = df.columns.swaplevel(0, 1)
df

In [ ]:
df['BTCUSDT']

In [ ]:
def recalculate_volumes(df, df_old):
    df.columns = df.columns.swaplevel(0, 1)
    df_old.columns = df_old.columns.swaplevel(0, 1)
    df = pd.concat([df, df_old[['base_volume', 'quote_volume']]], 
                   join='outer', axis='columns')
    df.columns = df.columns.swaplevel(0, 1)
    df = df.stack(level=1).reset_index(level=1)
    df[df['pair'] == 'base_volume'].iloc[:,1:] = \
        df[df['pair'] == 'rolling_base_volume'].iloc[:,1:].diff(1) + \
        df[df['pair'] == 'base_volume'].iloc[:,1:].shift(1440)
    df[df['pair'] == 'quote_volume'].iloc[:,1:] = \
        df[df['pair'] == 'rolling_quote_volume'].iloc[:,1:].diff(1) + \
        df[df['pair'] == 'quote_volume'].iloc[:,1:].shift(1440)
    df = df.reset_index().pivot_table(index=['date'], columns=['pair'], 
                                      values=df.columns[1:])
    df.columns = df.columns.swaplevel(0, 1)
    df = df[['open', 'high', 'low', 'close', 
             'base_volume', 'quote_volume', 
             'rolling_base_volume', 
             'rolling_quote_volume']]
    df.columns = df.columns.swaplevel(0, 1)
    return df

def resample(df, df_old):
    interval = '1min'
    df.index = pd.DatetimeIndex(df.index).round(interval)
    df = df.stack(level=0).reset_index(level=1)
    frequency = pd.tseries.frequencies.to_offset((df.index[1:] - df.index[:-1]).min())
    frequency_1min = pd.tseries.frequencies.to_offset('1min')
    if frequency > frequency_1min:
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['open', 'high', 'low', 'close', 
                                    'base_volume', 'quote_volume', 
                                    'rolling_base_volume', 
                                    'rolling_quote_volume'], 
                            aggfunc={'open': 'first', 'high': 'max', 
                                     'low': 'min', 'close': 'last', 
                                     'base_volume': 'sum', 
                                     'quote_volume': 'sum', 
                                     'rolling_base_volume': 'sum', 
                                     'rolling_quote_volume': 'sum'})
    else:
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['open', 'high', 'low', 'close', 
                                    'rolling_base_volume', 
                                    'rolling_quote_volume'], 
                            aggfunc={'open': 'first', 'high': 'max', 
                                     'low': 'min', 'close': 'last', 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})

    #if frequency == frequency_1min:
    #df = recalculate_volumes(df, df_old)

    df.columns = df.columns.swaplevel(0, 1)
    return df.sort_index()

#df_old = None
df1 = resample(df, df_old)
#df1.columns = df1.columns.swaplevel(0, 1)
#df1 = recalculate_volumes(df1, df_old)
#df1.columns = df1.columns.swaplevel(0, 1)
df1[symbol].dropna()

In [ ]:
df1[symbol].resample('30s').agg('max').fillna(method='pad')

In [ ]:
df1[df1['pair'] == 'quote_volume']['ZILBUSD'].dropna()

In [ ]:
df1[df1['pair'] == 'quote_volume']['ZILBUSD'].dropna().loc[:df['ZILBUSD']['quote_volume'].dropna().index[-1]].tail(10)

In [ ]:
df['ZILBUSD']['quote_volume'].dropna().tail(10)

In [ ]:
interval = '1min'
df1 = df.copy()
df1.index = pd.DatetimeIndex(df1.index).round(interval)
df1 = df1.stack(level=0).reset_index(level=1)
df1

In [ ]:
interval = '1min'
df1 = df.copy()
df1.index = pd.DatetimeIndex(df1.index).round(interval)
df1 = df1.stack(level=0).reset_index(level=1)
df1 = df1.pivot_table(index=['date'], columns=['symbol'], 
                      values=['open', 'high', 'low', 'close', 
                              'base_volume', 'quote_volume', 
                              'rolling_base_volume', 
                              'rolling_quote_volume'], 
                      aggfunc={'open': 'first', 'high': 'max', 
                               'low': 'min', 'close': 'last', 
                               'base_volume': 'sum', 
                               'quote_volume': 'sum', 
                               'rolling_base_volume': 'sum', 
                               'rolling_quote_volume': 'sum'})
#df1['base_volume'] = df1['base_volume'].fillna(0)
#df1['quote_volume'] = df1['quote_volume'].fillna(0)
#df1['rolling_base_volume'] = df1['rolling_base_volume'].fillna(0)
#df1['rolling_quote_volume'] = df1['rolling_quote_volume'].fillna(0)
df1.columns = df1.columns.swaplevel(0, 1)
df1

In [ ]:
df1['ZILBUSD']

In [ ]:
interval = '1min'
df.index = pd.DatetimeIndex(df.index).round(interval)
df = df.stack(level=0).reset_index(level=1)
frequency = pd.tseries.frequencies.to_offset(df.index[1] - df.index[0])
frequency_1min = pd.tseries.frequencies.to_offset('1min')
if frequency < frequency_1min:
    df = df.pivot_table(index=['date'], columns=['symbol'], 
                        values=['open', 'high', 'low', 'close', 
                                'rolling_base_volume', 
                                'rolling_quote_volume'], 
                        aggfunc={'open': 'first', 'high': 'max', 
                                 'low': 'min', 'close': 'last', 
                                 'rolling_base_volume': 'max', 
                                 'rolling_quote_volume': 'max'})
else:
    df = df.pivot_table(index=['date'], columns=['symbol'], 
                        values=['open', 'high', 'low', 'close', 
                                'base_volume', 'quote_volume', 
                                'rolling_base_volume', 
                                'rolling_quote_volume'], 
                        aggfunc={'open': 'first', 'high': 'max', 
                                 'low': 'min', 'close': 'last', 
                                 'base_volume': 'sum', 
                                 'quote_volume': 'sum', 
                                 'rolling_base_volume': 'sum', 
                                 'rolling_quote_volume': 'sum'})
    df['base_volume'] = df['base_volume'].fillna(0)
    df['quote_volume'] = df['quote_volume'].fillna(0)

df['rolling_base_volume'] = df['rolling_base_volume'].fillna(0)
df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(0)

if frequency == frequency_1min:
    df['base_volume'] = 

df = df.fillna(method='pad').fillna(method='backfill')
df.columns = df.columns.swaplevel(0, 1)
return df.sort_index()

In [ ]:
from binance.client import Client
client = Client()
df = pd.DataFrame(client.get_ticker())
df

In [ ]:
df[df['symbol'] == symbol]['volume']

In [ ]:
df[df['symbol'] == symbol]['quoteVolume']

In [ ]:
df = pd.DataFrame(client.get_ticker())
float(df[df['symbol'] == symbol]['volume'].iloc[-1]) * float(df[df['symbol'] == symbol]['lastPrice'].iloc[-1])

In [ ]:
float(df[df['symbol'] == symbol]['lastQty'].iloc[-1]) * float(df[df['symbol'] == symbol]['lastPrice'].iloc[-1])

In [ ]:
#symbol = 'NEBLBUSD'
from cryptocurrency.ohlcv import download_pair
df = download_pair(client, symbol, interval='1m', period=2000)
#df = df.loc[:df_1min.index[-1]]
df.tail(10)

In [ ]:
quote_volume = df['quote_volume'] * df['close']
quote_volume

In [ ]:
new_rolling_volume = quote_volume.rolling('1440min').sum()
new_rolling_volume.tail(10)

In [ ]:
new_standard_volume = (new_rolling_volume.diff(1) + quote_volume.shift(1440))
new_standard_volume.tail(10)

In [ ]:
import numpy as np
np.convolve((new_rolling_volume).rolling('1440min').agg(np.sum), np.array([1, 0]), 'same')

In [ ]:
from pandas import read_csv

crypto_output_log_15s = 'crypto_logs/crypto_output_log_15s.txt'
df = read_csv(crypto_output_log_15s, header=[0, 1], index_col=0)
df

In [ ]:
df['NEBLBUSD']['volume']

In [ ]:
import pandas as pd
pd.infer_freq(df.asfreq('1min').index)

In [ ]:
df.index.freq == pd.tseries.frequencies.to_offset('1min')

In [ ]:
df['BTCUSDT'].drop(columns=['volume']).iloc[-100:].plot(figsize=(20, 10))

In [ ]:
# Run this from local machine:
#apt update
#apt install -y openssh-server sshfs vde2
#mkdir -p /home/samuel/workspace/crypto_logs/
#dpipe /usr/lib/openssh/sftp-server = ssh sam@154.12.239.24 sshfs :/home/samuel/workspace/crypto_logs/ /home/sam/workspace/crypto_logs -o slave &

In [ ]:
import pandas as pd
input_filtered = pd.read_csv(crypto_logger_output_1min.input_log_screened_name, header=None, 
                             index_col=0, names=['date', 'symbol', 'lastPrice'])
input_filter = input_filtered['symbol']
#input_filter = set(input_filter.tolist())
input_filter.isin(df['baseAsset'])

In [ ]:
from pandas import read_csv

crypto_input_log_1min = 'crypto_logs/crypto_input_log_1min.txt'
df = pd.read_csv(crypto_input_log_1min, header=0, index_col=0)
df

In [ ]:
df.columns

In [ ]:
symbol = 'BTCUSDT'
interval = '1m'
period = 2880

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.ohlcv import download_pair

authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client

df = download_pair(client=client, symbol=symbol, interval=interval, period=period)
df

In [ ]:
import pandas as pd
df.astype('float32').memory_usage().sum()

In [ ]:
df.memory_usage().sum()